![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [2]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

In [19]:
# make copy 
df = ds_jobs_transformed.copy()

In [15]:
# print info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

**Two factor columns as boolean**

In [24]:
# identify columns with only two factors
for col in df.columns:
    if df[col].nunique() == 2:
        print(col)


# convert jrelevant experience to boolean dtype
#df['relevant_experience'] = df['relevant_experience'].map({'No relevant experience': 0, 'Has relevant experience': 1})
df['relevant_experience'] = df['relevant_experience'].astype('bool')

# convert job change to boolean dtype
df['job_change'] = df['job_change'].astype('bool')


relevant_experience
job_change


In [25]:
print(df['job_change'].unique())
print(df['relevant_experience'].unique())

# print counts of relevant_experience
print(df['relevant_experience'].value_counts())

[ True False]
[ True False]
relevant_experience
True     13792
False     5366
Name: count, dtype: int64


**integer columns only as 32-bit integers**

In [26]:
# print columns containing integers only
for col in df.columns:
    if df[col].dtype == 'int64':
        print(col)

student_id
training_hours


In [28]:
# convet student_id and training_hours to int32
df['student_id'] = df['student_id'].astype('int32')
df['training_hours'] = df['training_hours'].astype('int32')

**columns containing floats as float16**

In [30]:
for col in df.columns:
    if df[col].dtype == 'float':
        print(col)

city_development_index


In [32]:
# convert city_development_index to float16
df['city_development_index'] = df['city_development_index'].astype('float16')

**columns containing nominal categorical data as category data**

In [33]:
# print columns containing nominal categorical data
for col in df.columns:
    if df[col].dtype == 'object':
        print(col)

city
gender
enrolled_university
education_level
major_discipline
experience
company_size
company_type
last_new_job


In [34]:
# convert all nominal categorical data to category dtype
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category')

**ordinal category data to ordered categories**

In [36]:
# print categories of categorical data
for col in df.columns:
    if df[col].dtype == 'category':
        print(col)
        print(df[col].cat.categories)


city
Index(['city_1', 'city_10', 'city_100', 'city_101', 'city_102', 'city_103',
       'city_104', 'city_105', 'city_106', 'city_107',
       ...
       'city_84', 'city_89', 'city_9', 'city_90', 'city_91', 'city_93',
       'city_94', 'city_97', 'city_98', 'city_99'],
      dtype='object', length=123)
gender
Index(['Female', 'Male', 'Other'], dtype='object')
enrolled_university
Index(['Full time course', 'Part time course', 'no_enrollment'], dtype='object')
education_level
Index(['Graduate', 'High School', 'Masters', 'Phd', 'Primary School'], dtype='object')
major_discipline
Index(['Arts', 'Business Degree', 'Humanities', 'No Major', 'Other', 'STEM'], dtype='object')
experience
Index(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2',
       '20', '3', '4', '5', '6', '7', '8', '9', '<1', '>20'],
      dtype='object')
company_size
Index(['10-49', '100-499', '1000-4999', '10000+', '50-99', '500-999',
       '5000-9999', '<10'],
      dtype='object')
company_type
Inde

In [38]:
# order education_level categories in ascending order
df['education_level'] = df['education_level'].cat.reorder_categories(['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered=True)

# order company_size categories in ascending order
df['company_size'] = df['company_size'].cat.reorder_categories(['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'], ordered=True)

# order company_type categories in ascending order
df['company_type'] = df['company_type'].cat.reorder_categories(['Pvt Ltd', 'Funded Startup', 'Public Sector', 'Early Stage Startup', 'NGO', 'Other'], ordered=True)

# order last_new_job categories in ascending order
df['last_new_job'] = df['last_new_job'].cat.reorder_categories(['never', '1', '2', '3', '4', '>4'], ordered=True)

# order experience categories in ascending order
df['experience'] = df['experience'].cat.reorder_categories(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'], ordered=True)

# orderenrolled_university categories in ascending order
df['enrolled_university'] = df['enrolled_university'].cat.reorder_categories(['no_enrollment', 'Part time course', 'Full time course'], ordered=True)

In [46]:
ds_jobs_transformed = df.copy()

print(ds_jobs.memory_usage())


Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64


In [43]:
print(df.memory_usage())

Index                       128
student_id                76632
city                      24310
city_development_index    38316
gender                    19290
relevant_experience       19158
enrolled_university       19290
education_level           19370
major_discipline          19378
experience                19890
company_size              19522
company_type              19378
last_new_job              19378
training_hours            76632
job_change                19158
dtype: int64


In [49]:
print(ds_jobs_transformed.memory_usage())

Index                     17608
student_id                 8804
city                       7353
city_development_index     4402
gender                     2333
relevant_experience        2201
enrolled_university        2333
education_level            2413
major_discipline           2421
experience                 2933
company_size               2565
company_type               2421
last_new_job               2421
training_hours             8804
job_change                 2201
dtype: int64


**filtering**

In [47]:
# print experience categories
print(df['experience'].cat.categories)

Index(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '>20'],
      dtype='object')


In [48]:
#filter students with 10 or more years of experience  and at least 1000 employees
experience_10 = ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
company_size_1000 = ['1000-4999', '5000-9999', '10000+']
ds_jobs_transformed = df[(df['experience'].isin(experience_10)) & (df['company_size'].isin(company_size_1000))]

